In [1]:
%%capture
!pip install luigi pyyaml

In [2]:
%%capture
!pip install "xarray[complete]"==2023.8.0 s3fs --user

In [3]:
import os
import re
import glob
import xarray as xr
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
landing_path = "/home/ec2-user/SageMaker/datalake/data/agera/temperature/brozen/"
output_path = "/home/ec2-user/SageMaker/datalake/data/agera/temperature/convert"
filename = "Temperature-Air-2m-Max-24h_C3S-glob-agric_AgERA5_19790101_final-v1.0.nc"
var_name = "Temperature_Air_2m_Max_24h"

In [5]:
file_path = os.path.join(landing_path, filename)
ds = xr.open_dataset(file_path)

In [6]:
def kelvin_to_celsius(temp_k):
    return temp_k - 273.15

In [8]:
temp_var = ds[var_name]
temp_celsius = kelvin_to_celsius(temp_var.where(temp_var.notnull(), drop=True))
ds[var_name] = temp_celsius
ds.to_netcdf(os.path.join(output_path, filename))
ds.close()

## Proceso masivo

In [ ]:
def kelvin_to_celsius(temp_k):
    return temp_k - 273.15

def process_temperature_file(landing_path, filename, output_path,var_name):
    file_path = os.path.join(landing_path, filename)
    ds = xr.open_dataset(file_path)
    temp_var = ds[var_name]
    # Kelvin a Celsius
    #temp_celsius = kelvin_to_celsius(temp_var.where(temp_var.notnull(), drop=True)) # Excluir NAs
    temp_celsius = kelvin_to_celsius(temp_var.where(temp_var.notnull())) # No aplica conversión a los NAs
    ds[var_name] = temp_celsius
    output_file_path = os.path.join(output_path, filename)
    ds.to_netcdf(output_file_path)
    ds.close()

In [10]:
# Temperatura Max
pattern = re.compile(r'.*-Max-.*\.nc$')
all_files = glob.glob(f"{landing_path}/*")
matching_files = [file for file in all_files if pattern.match(file)]

print(f"Archivos encontrados: {len(matching_files)}")
for file in matching_files[:5]:
    print(file)

Archivos encontrados: 16436
Primeros 5 archivos:
/home/ec2-user/SageMaker/datalake/data/agera/temperature/brozen/Temperature-Air-2m-Max-24h_C3S-glob-agric_AgERA5_19951005_final-v1.0.nc
/home/ec2-user/SageMaker/datalake/data/agera/temperature/brozen/Temperature-Air-2m-Max-24h_C3S-glob-agric_AgERA5_20020425_final-v1.0.nc
/home/ec2-user/SageMaker/datalake/data/agera/temperature/brozen/Temperature-Air-2m-Max-24h_C3S-glob-agric_AgERA5_20171121_final-v1.0.nc
/home/ec2-user/SageMaker/datalake/data/agera/temperature/brozen/Temperature-Air-2m-Max-24h_C3S-glob-agric_AgERA5_19960402_final-v1.0.nc
/home/ec2-user/SageMaker/datalake/data/agera/temperature/brozen/Temperature-Air-2m-Max-24h_C3S-glob-agric_AgERA5_20011119_final-v1.0.nc


In [ ]:
def process_all_files(input_files, output_folder):
    for file in input_files:
        output_file = os.path.join(output_folder, os.path.basename(file))
        print(f"Procesando: {file}")
        process_temperature_file(file, output_file)
    print("Procesamiento completado.")

In [ ]:
process_all_files(matching_files, output_folder)

In [ ]:
# Validation
processed_files = glob.glob(f"{output_folder}/*.nc")
print(f"Archivos procesados: {len(processed_files)}")
print("Primeros 5 archivos procesados:")
for file in processed_files[:5]:
    print(file)

In [ ]:
# Comparar un archivo original con uno procesado
if matching_files and processed_files:
    original = xr.open_dataset(matching_files[0])
    processed = xr.open_dataset(processed_files[0])
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    original['temperature'].isel(time=0).plot()
    plt.title("Original (Kelvin)")
    plt.subplot(1, 2, 2)
    processed['temperature'].isel(time=0).plot()
    plt.title("Procesado (Celsius)")
    plt.tight_layout()
    plt.show()
    original.close()
    processed.close()

In [ ]:
import os
import sys
import luigi
#os.environ["SRC_CODE"] = "/home/ec2-user/SageMaker/datalake/src/"
#srcPath = os.environ["SRC_CODE"]
srcPath = os.environ.get("SRC_CODE", "/home/ec2-user/SageMaker/datalake/src/")
sys.path.append(srcPath)
from population.miscellanius import S3utilities

In [ ]:
"""
        self.confType = confType
        self.year = year
        self.month = month

"""
class downloadDataFromS3(luigi.Task):
    confType = luigi.Parameterd()
    year = luigi.Parameter()
    month = luigi.Parameter()
    
    def run(self):
        S3utilities
        S3utilities.downloadDataFromS3Filtered(
            self.confType,
            self.year,
            self.month
        )

if __name__ == "__main__":
    luigi.build([downloadDataFromS3(confType="agera5ArgumentsSolarFlux",
                                    year=2024,
                                    month="01")])

In [ ]:
help(S3utilities)

In [ ]:
help(luigi.build)